# R- and D-type Ionization Fronts

Figure 4.10 from Chapter 4 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Plot density-speed ($\rho_2/\rho_1$ vs $u_1/c_1$) diagram for R- and D-type ionization fronts with annotation.

We do the calculations analytically in the notebook.

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

# Filter nuisance warnings

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)
warnings.filterwarnings('ignore',category=RuntimeWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig4_10' 

# graphic aspect ratio = width/height

aspect = 4.0/3.0 # 4:3

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 8
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'


## Ionization front calculations

Sound speed $c_2$ is set to 10 times the pre-shock sound speed $c_1$.  We compute the speeds and densities
at the R- and D-critical points:
 * $v_R = c_2 + \sqrt{c_2^2-c_1^2}$
 * $v_D = c_2 - \sqrt{c_2^2-c_1^2}$
 * $\rho_{R,crit} = 0.5\rho_1\left(\frac{1+(v_R/c_1)^2}{c_2^2}\right)$
 * $\rho_{D,crit} = 0.5\rho_1\left(\frac{1+(v_D/c_1)^2}{c_2^2}\right)$
 
The physics behind these equations is described in section 4.5 of the book.

In [ ]:
# Sound speed c2 is 10x the pre-shock sound speed c1

c2 = 10.0

# Speeds (u1/c1) and density (rho2/rho1) at the R- and D-critical points

uRc = c2 + math.sqrt(c2*c2-1.)
uDc = c2 - math.sqrt(c2*c2-1.)

rhoRcrit = 0.5*(1.+uRc*uRc)/(c2*c2)
rhoDcrit = 0.5*(1.+uDc*uDc)/(c2*c2)

# R-type fronts

uMax = 100.*uRc

# compute on a logarithmic range of u1

loguMin = math.log10(uRc)
loguMax = math.log10(uMax)
uR = 10.0**np.linspace(loguMin,loguMax,501)

f = (uR*uR - uRc*uRc)*(uR*uR - uDc*uDc)

rhoRs = 0.5*(1+uR*uR + np.sqrt(f))/(c2*c2)
rhoRw = 0.5*(1+uR*uR - np.sqrt(f))/(c2*c2)

# D-type fronts

uMin = uDc/100.0
loguMin = math.log10(uMin)
loguMax = math.log10(uDc)
uD = 10.0**np.linspace(loguMin,loguMax,501)

f = (uRc*uRc - uD*uD)*(uDc*uDc - uD*uD)

rhoDs = 0.5*(1.+uD*uD - np.sqrt(f))/(c2*c2)
rhoDw = 0.5*(1.+uD*uD + np.sqrt(f))/(c2*c2)

# plotting limits

xMin = np.min(uD)
xMax = np.max(uR)

yMin = 1.0e-7 # np.min(rhoDs)
yMax = 1.0e5  # np.max(rhoRs)


### Make the Plot

Plot the physically permitted values of the density contrast $\rho_2/\rho_1$ across
an ionization front with $c_2=10c_1$.

In [ ]:
fig,ax = plt.subplots()

fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

# x-axis is log10(n_e)

plt.xlim(xMin,xMax)
ax.set_xscale('log')
ax.set_xticks([1e-3,1e-2,0.1,1,10,1e2,1e3])
plt.xlabel(r'u$_{\rm 1}$/c$_{\rm 1}$')

plt.ylim(yMin,yMax)
ax.set_yscale('log')
ax.set_yticks([1e-7,1e-6,1e-5,1e-4,1e-3,0.01,0.1,1,10,100,1e3,1e4,1e5])
plt.ylabel(r'$\rho_2/\rho_1$',fontsize=axisFontSize)

ax.xaxis.set_major_locator(matplotlib.ticker.LogLocator(base=10.0))
ax.yaxis.set_major_locator(matplotlib.ticker.LogLocator(base=10.0))

# R-front curve

plt.plot(uR,rhoRs,color='black',ls='-',lw=1.5,zorder=8)
plt.plot(uR,rhoRw,color='black',ls='-',lw=1.5,zorder=8)

# D-front curve

plt.plot(uD,rhoDs,color='black',ls='-',lw=1.5,zorder=8)
plt.plot(uD,rhoDw,color='black',ls='-',lw=1.5,zorder=8)

# Plot the R- and D-critical points

plt.plot([uRc,uDc],[rhoRcrit,rhoDcrit],'o',mfc='white',mec='black',mew=1,markersize=6,zorder=10)

# Annotation

plt.text(0.065,rhoDcrit,'D-critical',va='center',ha='left',fontsize=labelFontSize)
plt.text(0.005,0.012,'weak\nD',va='bottom',ha='center',fontsize=labelFontSize)
plt.text(0.016,5.0e-5,'strong\nD',va='top',ha='center',fontsize=labelFontSize)

plt.text(15.,rhoRcrit,'R-critical',va='center',ha='right',fontsize=labelFontSize)
plt.text(240.,0.65,'weak\nR',va='top',ha='center',fontsize=labelFontSize)
plt.text(175.,450.,'strong\nR',va='bottom',ha='center',fontsize=labelFontSize)

plt.text(0.01,100.,r'$c_2 = 10 c_1$',va='center',ha='left',fontsize=14)

# plot and file

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')